# Data Cleaning
---

## Now that we have web scraped our user reviews for the top 100 games on each console let's read in our data and combine it all into one master data frame.

In [1]:
import pandas as pd
from cleantext import clean

In [2]:
ps4_reviews = pd.read_csv('../data/top_100_ps4_games.csv')
xboxone_reviews = pd.read_csv('../data/top_100_xboxone_games.csv')
switch_reviews = pd.read_csv('../data/top_100_switch_games.csv')
pc_reviews = pd.read_csv('../data/top_100_pc_games.csv')
xbox_series_x_reviews = pd.read_csv('../data/top_100_xbox-series-x_games.csv')
ps5_reviews = pd.read_csv('../data/top_100_ps5_games.csv')

In [3]:
all_console_reviews = pd.concat(
    [ps4_reviews, xboxone_reviews, switch_reviews, pc_reviews, xbox_series_x_reviews, ps5_reviews], 
    ignore_index=True)

### Now that we have a master data frame let's take a look at the first few rows and get the shape to see how many total rows and columns we are working with.

In [4]:
print(all_console_reviews.shape)
all_console_reviews.head()

(112345, 11)


,console,video_game_name,summary,developer,genre(s),num_players,esrb_rating,critic_score,avg_user_score,user_review,user_score
0,ps4,Red Dead Redemption 2,Developed by the creators of Grand Theft Auto ...,Rockstar Games,"Genre(s): Action Adventure, Open-World",# of players: Up to 32,Rating: M,97,8.6,"\nThis site is a joke, this the first time whe...",9
1,ps4,Red Dead Redemption 2,Developed by the creators of Grand Theft Auto ...,Rockstar Games,"Genre(s): Action Adventure, Open-World",# of players: Up to 32,Rating: M,97,8.6,Fair review of RDR2\r I'm almost 15% finished ...,7
2,ps4,Red Dead Redemption 2,Developed by the creators of Grand Theft Auto ...,Rockstar Games,"Genre(s): Action Adventure, Open-World",# of players: Up to 32,Rating: M,97,8.6,I really wanted to love it. The over-world is ...,6
3,ps4,Red Dead Redemption 2,Developed by the creators of Grand Theft Auto ...,Rockstar Games,"Genre(s): Action Adventure, Open-World",# of players: Up to 32,Rating: M,97,8.6,"\nBeautiful graphics, excellent voice acting, ...",7
4,ps4,Red Dead Redemption 2,Developed by the creators of Grand Theft Auto ...,Rockstar Games,"Genre(s): Action Adventure, Open-World",# of players: Up to 32,Rating: M,97,8.6,This game is really overrated.\rThe amazing en...,7


## Now let's see how many null values are present and address them
---

In [5]:
all_console_reviews.isnull().sum()

console               0
video_game_name       0
summary               0
developer             0
genre(s)              0
num_players        8659
esrb_rating        6513
critic_score          0
avg_user_score     2729
user_review           0
user_score            0
dtype: int64

### Let's also look at what the datatypes are for the columns.

In [6]:
all_console_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112345 entries, 0 to 112344
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   console          112345 non-null  object 
 1   video_game_name  112345 non-null  object 
 2   summary          112345 non-null  object 
 3   developer        112345 non-null  object 
 4   genre(s)         112345 non-null  object 
 5   num_players      103686 non-null  object 
 6   esrb_rating      105832 non-null  object 
 7   critic_score     112345 non-null  int64  
 8   avg_user_score   109616 non-null  float64
 9   user_review      112345 non-null  object 
 10  user_score       112345 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 9.4+ MB


## First let's take a look at the `num_players` column, we want to clean the values and only get the number, if applicable. Then address the null values.

In [7]:
all_console_reviews['num_players'].unique()

array(['# of players: Up to 32', '# of players: Up to 30',
       '# of players: No Online Multiplayer', '# of players: Up to 8',
       '# of players: Up to 16', '# of players: Up to 10',
       '# of players: 2', '# of players: Up to 5',
       '# of players: Up to 4', nan, '# of players: 1 Player',
       '# of players: Up to 12', '# of players: Up to 6',
       '# of players: Massively Multiplayer', '# of players: Up to 60',
       '# of players: Online Multiplayer', '# of players: Up to 64',
       '# of players: Up to 3', '# of players: Up to 22',
       '# of players: Up to 20', '# of players: Up to 24',
       '# of players: Up to more than 64', '# of players: 1-16',
       '# of players: 1-32', '# of players: Up to 18', '# of players:',
       '# of players: 1-8', '# of players: 1-2', '# of players: 1-4',
       '# of players: 64 Online', '# of players: 2 Online',
       '# of players: Up to 14', '# of players: Up to 40',
       '# of players: Friend System Only'], dtype=objec

### Replace text values to only get the number of players, for games without a simple number investigate further

In [8]:
all_console_reviews['num_players'] = \
[val.replace('# of players: ', '').replace('Up to ', '').replace(' Player', '').replace('more than ', '')
 if type(val) != float else val for val in all_console_reviews['num_players']]

In [9]:
vals = []
for val in all_console_reviews['num_players']:
    if type(val) != float:
        if '-' in val:
            vals.append(val.split('-')[1])
        else:
            vals.append(val)
    else:
        vals.append(val)

all_console_reviews['num_players'] = vals

In [10]:
all_console_reviews['num_players'].unique()

array(['32', '30', 'No Online Multiplayer', '8', '16', '10', '2', '5',
       '4', nan, '1', '12', '6', 'Massively Multiplayer', '60',
       'Online Multiplayer', '64', '3', '22', '20', '24', '18',
       '# of players:', '64 Online', '2 Online', '14', '40',
       'Friend System Only'], dtype=object)

### Now that we did some initial cleaning let's look closer at games with the unique `num_players`
- No Online Multiplayer
- Massively Multiplayer
- Online Multiplayer
- \# of players:
- Friend System Only

In [11]:
list(all_console_reviews[all_console_reviews['num_players'] == 'No Online Multiplayer']\
['video_game_name'].unique())[:10]

['Persona 5 Royal',
 'God of War',
 'The Last of Us Part II',
 'Persona 5',
 'Undertale',
 'The Witcher 3: Wild Hunt',
 'Shadow of the Colossus',
 'The Witcher 3: Wild Hunt - Blood and Wine',
 'Celeste',
 'NieR: Automata - Game of the YoRHa Edition']

### For 'No Online Multiplayer' we can see that all of the games are single player games so we can change the num_players to 1.

In [12]:
all_console_reviews['num_players'] = \
[val.replace('No Online Multiplayer', '1')
 if type(val) != float else val for val in all_console_reviews['num_players']]

In [13]:
all_console_reviews['num_players'].unique()

array(['32', '30', '1', '8', '16', '10', '2', '5', '4', nan, '12', '6',
       'Massively Multiplayer', '60', 'Online Multiplayer', '64', '3',
       '22', '20', '24', '18', '# of players:', '64 Online', '2 Online',
       '14', '40', 'Friend System Only'], dtype=object)

In [14]:
list(all_console_reviews[all_console_reviews['num_players'] == 'Massively Multiplayer']\
['video_game_name'].unique())[:]

['Final Fantasy XIV: Stormblood',
 'World of Warcraft',
 'World of Warcraft: Wrath of the Lich King',
 'World of Warcraft: The Burning Crusade',
 'World of Warcraft: Cataclysm',
 'Final Fantasy XIV: Endwalker']

### For 'Massively Multiplayer' games which we can see are MMOs, massive multiplayer online games, they can have player counts in the millions. To keep our values within a reasonable range we will change these to a value of 150.

In [15]:
all_console_reviews['num_players'] = \
[val.replace('Massively Multiplayer', '150')
 if type(val) != float else val for val in all_console_reviews['num_players']]

In [16]:
all_console_reviews['num_players'].unique()

array(['32', '30', '1', '8', '16', '10', '2', '5', '4', nan, '12', '6',
       '150', '60', 'Online Multiplayer', '64', '3', '22', '20', '24',
       '18', '# of players:', '64 Online', '2 Online', '14', '40',
       'Friend System Only'], dtype=object)

In [17]:
list(all_console_reviews[all_console_reviews['num_players'] == 'Online Multiplayer']\
['video_game_name'].unique())[:]

['Dreams',
 'Divinity: Original Sin II - Definitive Edition',
 'Apex Legends',
 'Thronebreaker: The Witcher Tales',
 'Warframe',
 'Kingdom Two Crowns',
 'Half-Life',
 "Sid Meier's Civilization IV",
 'Microsoft Flight Simulator',
 'Spelunky 2',
 'Factorio',
 'The Pathless']

### For 'Online Multiplayer' these have varying number of players, some are MMOs, others battle royales, and others merely having a simple deathmatch mode. For simplicity sake we will change the value to 100, lower than 'Massively Multiplayer' since these were not labeled as the same.

In [18]:
all_console_reviews['num_players'] = \
[val.replace('Online Multiplayer', '100')
 if type(val) != float else val for val in all_console_reviews['num_players']]

In [19]:
all_console_reviews['num_players'].unique()

array(['32', '30', '1', '8', '16', '10', '2', '5', '4', nan, '12', '6',
       '150', '60', '100', '64', '3', '22', '20', '24', '18',
       '# of players:', '64 Online', '2 Online', '14', '40',
       'Friend System Only'], dtype=object)

In [20]:
list(all_console_reviews[all_console_reviews['num_players'] == '# of players:']\
['video_game_name'].unique())[:]

['The Sims']

### The Sims is a single player game with multiplayer aspects that really only revolve on downloading other players creations etc. We will change this to a value of 1.

In [21]:
all_console_reviews['num_players'] = \
[val.replace('# of players:', '1')
 if type(val) != float else val for val in all_console_reviews['num_players']]

In [22]:
all_console_reviews['num_players'].unique()

array(['32', '30', '1', '8', '16', '10', '2', '5', '4', nan, '12', '6',
       '150', '60', '100', '64', '3', '22', '20', '24', '18', '64 Online',
       '2 Online', '14', '40', 'Friend System Only'], dtype=object)

In [23]:
list(all_console_reviews[all_console_reviews['num_players'] == 'Friend System Only']\
['video_game_name'].unique())[:]

['Control: Ultimate Edition']

### Control is a single player game, change value to 1.

In [24]:
all_console_reviews['num_players'] = \
[val.replace('Friend System Only', '1')
 if type(val) != float else val for val in all_console_reviews['num_players']]

In [25]:
all_console_reviews['num_players'].unique()

array(['32', '30', '1', '8', '16', '10', '2', '5', '4', nan, '12', '6',
       '150', '60', '100', '64', '3', '22', '20', '24', '18', '64 Online',
       '2 Online', '14', '40'], dtype=object)

### Now we can do a final cleanup of values that have 'Online' and we are good to further inspect null values

In [26]:
all_console_reviews['num_players'] = \
[val.replace(' Online', '')
 if type(val) != float else val for val in all_console_reviews['num_players']]

In [27]:
all_console_reviews['num_players'].unique()

array(['32', '30', '1', '8', '16', '10', '2', '5', '4', nan, '12', '6',
       '150', '60', '100', '64', '3', '22', '20', '24', '18', '14', '40'],
      dtype=object)

In [28]:
all_console_reviews[all_console_reviews['num_players'].isnull()]['video_game_name'].unique()

array(['Final Fantasy XIV: Shadowbringers', 'INSIDE', 'Shovel Knight',
       'Tales From The Borderlands: Episode 5 - The Vault of the Traveler',
       'Monster Hunter: World - Iceborne', 'Rez Infinite', 'Bastion',
       'The Binding of Isaac: Rebirth',
       'Keep Talking and Nobody Explodes',
       'The Talos Principle: Deluxe Edition',
       'Nex Machina: Death Machine',
       'Guacamelee! Super Turbo Championship Edition',
       'Bloodborne: The Old Hunters', 'The Witness', 'Night in the Woods',
       'TowerFall Ascension', 'Psychonauts 2', 'NBA 2K17',
       'Destiny: The Taken King', 'Pro Evolution Soccer 2017',
       'Forza Horizon 3: Hot Wheels', 'DiRT Rally',
       'Killer Instinct Season 3', 'Destiny 2: Forsaken',
       'Dying Light: The Following', 'DiRT 4', 'Firewatch',
       'Tetris Effect: Connected', 'Chicory: A Colorful Tale',
       "Death's Door", 'DUSK', 'Portal 2', 'Warcraft III: Reign of Chaos',
       "Sid Meier's Alpha Centauri", 'Final Fantasy XIV: 

### The list of the games above with a missing `num_players` value range from indie developed games to massive AAA franchise games. A quick spot check of a few games shows that they are single player games. For this reason we will replace all null values in this column with 1. We will also change the type into an int.

In [29]:
all_console_reviews['num_players'].fillna('1', inplace=True)

In [30]:
all_console_reviews['num_players'] = all_console_reviews['num_players'].astype(int)

In [31]:
all_console_reviews.isnull().sum()

console               0
video_game_name       0
summary               0
developer             0
genre(s)              0
num_players           0
esrb_rating        6513
critic_score          0
avg_user_score     2729
user_review           0
user_score            0
dtype: int64

## Now let's take a look at the `esrb_rating` column, we want to clean the values and then address the null values. For further information on ESRB rating see [ESRB Rating Guide](https://www.esrb.org/ratings-guide/)

In [32]:
all_console_reviews['esrb_rating'].unique()

array(['Rating: M', 'Rating: T', 'Rating: E', 'Rating: E10+', nan,
       'Rating: K-A'], dtype=object)

### We want to extract only the rating itself and get rid of any extra text.

In [33]:
all_console_reviews['esrb_rating'] = \
[val.replace('Rating: ', '')
 if type(val) != float else val for val in all_console_reviews['esrb_rating']]

In [34]:
all_console_reviews['esrb_rating'].unique()

array(['M', 'T', 'E', 'E10+', nan, 'K-A'], dtype=object)

### Let's take a closer look at 'K-A'.

In [35]:
list(all_console_reviews[all_console_reviews['esrb_rating'] == 'K-A']\
['video_game_name'].unique())

["Sid Meier's Civilization II", "Sid Meier's Gettysburg!"]

### Looking into these games and the ['K-A' rating history](https://www.esrb.org/history/) we can safely change the value to E.

In [36]:
all_console_reviews['esrb_rating'] = \
[val.replace('K-A', 'E')
 if type(val) != float else val for val in all_console_reviews['esrb_rating']]

In [37]:
all_console_reviews['esrb_rating'].unique()

array(['M', 'T', 'E', 'E10+', nan], dtype=object)

### Now that we have cleaned up our values let's take a closer look at the null values

In [38]:
all_console_reviews[all_console_reviews['esrb_rating'].isnull()]['video_game_name'].unique()

array(['Injustice 2: Legendary Edition', 'Pistol Whip',
       'Forza Horizon 3: Hot Wheels', 'Killer Instinct Season 3',
       'Forza Horizon 2', 'Guacamelee! 2', 'Tetris Effect: Connected',
       'Chicory: A Colorful Tale', 'Fez',
       'Guacamelee! Super Turbo Championship Edition', 'Sumire',
       'Half-Life: Alyx', 'Divinity: Original Sin II', 'Beat Saber',
       'Undertale', 'Galactic Civilizations II: Twilight of the Arnor',
       'Final Fantasy XIV: Endwalker', 'Out of the Park Baseball 17',
       'Kentucky Route Zero - Act III',
       'Galactic Civilizations II: Dark Avatar', 'Disco Elysium',
       'Spelunky 2', 'Factorio', 'Spelunky',
       'The Stanley Parable: Ultra Deluxe',
       "Tony Hawk's Pro Skater 1 + 2", 'Planet Coaster: Console Edition',
       "Assassin's Creed Valhalla", "Marvel's Guardians of the Galaxy",
       'Yakuza: Like a Dragon', 'MLB The Show 21', 'NBA 2K21',
       'Football Manager 2021', 'Haven', 'Sniper Elite 5', 'Exo One',
       'Samurai

In [39]:
all_console_reviews['esrb_rating'].describe()

count     105832
unique         4
top            M
freq       53687
Name: esrb_rating, dtype: object

### The list of games with a null rating are a variety of different games, but for simplicity let us impute the null values to be the most frequent rating, which in this case is the M rating.

In [40]:
all_console_reviews['esrb_rating'].fillna('M', inplace=True)

In [41]:
all_console_reviews.isnull().sum()

console               0
video_game_name       0
summary               0
developer             0
genre(s)              0
num_players           0
esrb_rating           0
critic_score          0
avg_user_score     2729
user_review           0
user_score            0
dtype: int64

## Now let's take a look at the `avg_user_score` column, we want to clean the values and then address the null values.

In [42]:
all_console_reviews['avg_user_score'].describe()

count    109616.000000
mean          8.004043
std           1.215970
min           2.000000
25%           7.700000
50%           8.400000
75%           8.800000
max           9.400000
Name: avg_user_score, dtype: float64

In [43]:
all_console_reviews['avg_user_score'].unique()

array([8.6, 8.3, 8.5, 9.2, 9.1, 5.7, 8.7, 8.2, 8.8, 8.9, 7.2, 7.9, 8.4,
       7.8, 7.6, 9. , 8. , 7.3, 6.4, 7.7, 7.5, 6.9, 7.4, 6.3, 7. , 6.6,
       7.1, 6.1, 8.1, 6.5, 5.2, 6.7, 6.8, 5.9, 5.3, 5.6, 5.8, 4.4, 6.2,
       3.4, 4.2, 4.1, 9.4, 3.3, 5. , 6. , 3.8, nan, 5.4, 5.5, 3.6, 5.1,
       3.5, 4.8, 2.8, 4.6, 3.9, 4.7, 4.5, 2. , 2.5, 2.6])

### We will filter the df by games with null in avg_user_score. Then group by video_game_name and impute nulls with the mean user score for each of those games.

In [44]:
null_avg_user_score = all_console_reviews[all_console_reviews['avg_user_score'].isnull()]
vg_groupby_df = null_avg_user_score.groupby('video_game_name').mean()

In [45]:
vg_groupby_df

,num_players,critic_score,avg_user_score,user_score
video_game_name,,,,
Death Stranding: Director's Cut,1.0,85.0,NaN,8.288660
Forza Horizon 5,1.0,92.0,NaN,7.759000
Horizon Forbidden West,1.0,88.0,NaN,8.596000
Lawn Mowing Simulator,1.0,72.0,NaN,4.000000
Marvel's Guardians of the Galaxy,1.0,80.0,NaN,8.588477
MotoGP 21,14.0,77.0,NaN,9.000000
MotoGP 22,12.0,81.0,NaN,10.000000
Relayer,1.0,77.0,NaN,9.000000
Roki,1.0,82.0,NaN,9.000000


### Create a dictionary from the games with null avg_user_score and then use the video game names as keys and mean user_score as values.

In [46]:
user_score_avg_impute = {game:vg_groupby_df.iloc[i,3] for i, game in enumerate(list(vg_groupby_df.index))}

In [47]:
user_score_avg_impute

{"Death Stranding: Director's Cut": 8.288659793814434,
 'Forza Horizon 5': 7.759,
 'Horizon Forbidden West': 8.596,
 'Lawn Mowing Simulator': 4.0,
 "Marvel's Guardians of the Galaxy": 8.588477366255145,
 'MotoGP 21': 9.0,
 'MotoGP 22': 10.0,
 'Relayer': 9.0,
 'Roki': 9.0,
 'The Quarry': 6.285714285714286}

### Finally lets replace the null values with the values in the dictionary.

In [48]:
for key in user_score_avg_impute:
    all_console_reviews.loc[all_console_reviews.video_game_name == key, 'avg_user_score']\
    = round(user_score_avg_impute[key], 1)

In [49]:
all_console_reviews.isnull().sum()

console            0
video_game_name    0
summary            0
developer          0
genre(s)           0
num_players        0
esrb_rating        0
critic_score       0
avg_user_score     0
user_review        0
user_score         0
dtype: int64

## We have successfully removed all null values!

## Now let's clean up our `summary` column.

In [50]:
all_console_reviews['summary'].unique()[:3]

array(['Developed by the creators of Grand Theft Auto V and Red Dead Redemption, Red Dead Redemption 2 is an epic tale of life in America’s unforgiving heartland. The game’s vast and atmospheric world also provides the foundation for a brand new online multiplayer experience. America, 1899. The end of the Wild West era has begun. After a robbery goes badly wrong in the western town of Blackwater, Arthur Morgan and the Van der Linde gang are forced to flee. With federal agents and the best bounty hunters in the nation massing on their heels, the gang has to rob, steal and fight their way across the rugged heartland of America in order to survive. As deepening internal fissures threaten to tear the gang apart, Arthur must make a choice between his own ideals and loyalty to the gang that raised him. [Rockstar]',
       'The sprawling sun-soaked metropolis of Los Santos is chock full of self-help coaches, starlets and C-List celebrities, once on top of the media world, now struggling to st

### Let's use the `clean` function to clean up the video game summaries.

In [51]:
all_console_reviews['summary'] = \
[clean(review, no_emoji=True, no_line_breaks=True, no_urls=True, no_emails=True, no_numbers=True, no_phone_numbers=True, 
no_currency_symbols=True, no_punct=True) for review in all_console_reviews['summary']]

In [52]:
all_console_reviews['summary'].unique()[:3]

array(['developed by the creators of grand theft auto v and red dead redemption red dead redemption <number> is an epic tale of life in americas unforgiving heartland the games vast and atmospheric world also provides the foundation for a brand new online multiplayer experience america <number> the end of the wild west era has begun after a robbery goes badly wrong in the western town of blackwater arthur morgan and the van der linde gang are forced to flee with federal agents and the best bounty hunters in the nation massing on their heels the gang has to rob steal and fight their way across the rugged heartland of america in order to survive as deepening internal fissures threaten to tear the gang apart arthur must make a choice between his own ideals and loyalty to the gang that raised him rockstar',
       'the sprawling sunsoaked metropolis of los santos is chock full of selfhelp coaches starlets and clist celebrities once on top of the media world now struggling to stay relevant 

## Now let's clean up our `genre(s)` column.

In [53]:
all_console_reviews['genre(s)'].unique()[:10]

array(['Genre(s): Action Adventure, Open-World',
       'Genre(s): Action Adventure, Modern, Open-World',
       'Genre(s): Role-Playing, Japanese-Style',
       'Genre(s): Action Adventure, General, Modern',
       'Genre(s): Action Adventure, Linear',
       'Genre(s): General, Action Adventure, Survival',
       'Genre(s): Modern, Action Adventure, Modern, Open-World',
       'Genre(s): General, Modern, Action Adventure, Linear',
       'Genre(s): Action, Platformer, 3D',
       'Genre(s): Fantasy, Role-Playing, Action Adventure, General, Action RPG'],
      dtype=object)

### We only want the actual genres so replace the extra text.

In [54]:
all_console_reviews['genre(s)'] = [genre.replace('Genre(s): ', '') for genre in all_console_reviews['genre(s)']]

In [55]:
all_console_reviews['genre(s)'].unique()[:10]

array(['Action Adventure, Open-World',
       'Action Adventure, Modern, Open-World',
       'Role-Playing, Japanese-Style',
       'Action Adventure, General, Modern', 'Action Adventure, Linear',
       'General, Action Adventure, Survival',
       'Modern, Action Adventure, Modern, Open-World',
       'General, Modern, Action Adventure, Linear',
       'Action, Platformer, 3D',
       'Fantasy, Role-Playing, Action Adventure, General, Action RPG'],
      dtype=object)

## Finally using the `clean` function let's clean up our user reviews

In [56]:
all_console_reviews['user_review'].unique()[:2]

array(["\nThis site is a joke, this the first time when i see the critics getting the game right and the users getting it wrong. Either people are trolling or just brain dead. This game is a game of the year contender and y'all say its mediocre. The disrespect, i can't take this site serious anymore.\n",
       "Fair review of RDR2\r I'm almost 15% finished the game overall, which might be a little premature to review, but I just don't know if I can be bothered finishing the game, in all honesty. The things that are bothering me about this game are things that are fundamental to the gameplay anyway, and won't change no matter how much longer I play it.\r How to sum up RDR2: It's really not that much fun to play, sadly. Considering I paid a fair sum for it, and am a big RDR, and was waiting ages in anticipation of this game's release, then I ask myself why my PS4 is switched off and I'd rather surf the net instead? I should be glued to the game during any free hours that are available, 

### We can see that theres a lot of mess in this reviews, let's use the clean function to clean them up!

In [57]:
all_console_reviews['user_review'] = \
[clean(review, no_emoji=True, no_line_breaks=True, no_urls=True, no_emails=True, no_numbers=True, no_phone_numbers=True, 
no_currency_symbols=True, no_punct=True) for review in all_console_reviews['user_review']]

In [58]:
all_console_reviews['user_review'].unique()[:2]

array(['this site is a joke this the first time when i see the critics getting the game right and the users getting it wrong either people are trolling or just brain dead this game is a game of the year contender and yall say its mediocre the disrespect i cant take this site serious anymore',
       'fair review of rdr2 im almost <number> finished the game overall which might be a little premature to review but i just dont know if i can be bothered finishing the game in all honesty the things that are bothering me about this game are things that are fundamental to the gameplay anyway and wont change no matter how much longer i play it how to sum up rdr2 its really not that much fun to play sadly considering i paid a fair sum for it and am a big rdr and was waiting ages in anticipation of this games release then i ask myself why my ps4 is switched off and id rather surf the net instead i should be glued to the game during any free hours that are available chomping at the bit to get back

## With our data frame now cleaned, we will save it to a final .csv file

In [59]:
all_console_reviews.to_csv('../data/cleaned_all_console_reviews.csv', index=False)

In [60]:
all_console_reviews.head()

,console,video_game_name,summary,developer,genre(s),num_players,esrb_rating,critic_score,avg_user_score,user_review,user_score
0,ps4,Red Dead Redemption 2,developed by the creators of grand theft auto ...,Rockstar Games,"Action Adventure, Open-World",32,M,97,8.6,this site is a joke this the first time when i...,9
1,ps4,Red Dead Redemption 2,developed by the creators of grand theft auto ...,Rockstar Games,"Action Adventure, Open-World",32,M,97,8.6,fair review of rdr2 im almost <number> finishe...,7
2,ps4,Red Dead Redemption 2,developed by the creators of grand theft auto ...,Rockstar Games,"Action Adventure, Open-World",32,M,97,8.6,i really wanted to love it the overworld is be...,6
3,ps4,Red Dead Redemption 2,developed by the creators of grand theft auto ...,Rockstar Games,"Action Adventure, Open-World",32,M,97,8.6,beautiful graphics excellent voice acting lots...,7
4,ps4,Red Dead Redemption 2,developed by the creators of grand theft auto ...,Rockstar Games,"Action Adventure, Open-World",32,M,97,8.6,this game is really overrated the amazing envi...,7


## Data Dictionary
|Feature|Type|Description|
|---|---|---|
| console | object |electronic device that outputs a video signal or image to display a video game|
| video_game_name |object | name of video game|
| summary |object | summary of what video game is about|
| developer |object | the developer (creator) of the video game|
| genre(s) |object | genre of video game|
| num_players* | int| max number of players that can play in the same game environment at the same time|
| esrb_rating |object | ESRB (Entertainment Software Rating Board) rating of video game|
| critic_score |int| critic review score of video game|
| avg_user_score |float| average user review score of video game|
| user_review |object | user review of video game|
| user_score |int|user score of video game |

\* values of 100 - games with online multiplayer <br>
\* values of 150 - games considered to be MMOs (massive multipler online games)